In [1]:
#!/usr/bin/env python
# encoding: utf-8

In [2]:
import csv
import pandas as pd 
import numpy as np 
from collections import defaultdict
import re
import sys
from sklearn.model_selection import train_test_split
import datetime
import matplotlib.pyplot as plt
import time
import numpy as np
from pprint import pprint

import spacy  # For preprocessing
import nltk
import string
from nltk.corpus import stopwords
import preprocessor as p  #pip install tweet-preprocessor
import logging  # Setting up the loggings to monitor gensim
from nltk.stem import WordNetLemmatizer
from string import punctuation as punc

import gensim
import gensim.corpora as corpora
#from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.models as gsm
from gensim.test.utils import datapath

import regex
import emoji #pip install emoji --upgrade
# Internal dependencies
import word_emoji2vec as we2v
#from word_emoji2vec import Word_Emoji2Vec

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torchvision.transforms as transforms
from torch.autograd import Variable

nlp = spacy.load('en', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed #python -m spacy download en

torch.manual_seed(1)

In [3]:
# Load pre-trained emoji2vec 

e2v = gsm.KeyedVectors.load_word2vec_format('pre-trained/emoji2vec.bin', binary=True)
#happy_emoji_vector = e2v['🇯🇵']    # Produces an embedding vector of length 300
#w2v = gsm.KeyedVectors.load_word2vec_format('data/word2vec/w2v_model')
#happy_word_vector = w2v ['happy']
#print(happy_emoji_vector)
#print(happy_word_vector)
#vec = we2v['I am really happy right now! 😄']
#vec = Word_Emoji2Vec.from_word2vec_paths['love life 😄']
#print(vec)
#happy_word_emoji_vector = np.concatenate((happy_word_vector, happy_emoji_vector), axis=0)
#print(happy_word_emoji_vector, len(happy_word_emoji_vector), happy_word_emoji_vector.shape)#600 (600,)

In [4]:
#Load pre-trained word2vec

#w2v = gsm.KeyedVectors.load_word2vec_format("pre-trained/numberbatch-19.08.txt.gz") #problem need to fix
w2v = gsm.KeyedVectors.load_word2vec_format('pre-trained/GoogleNews-vectors-negative300.bin', binary=True) #ok
#happy_word_vector = w2v ['happy']
#print(happy_word_vector)

In [5]:
# weights to use in from_pretrained()
weights_text = torch.FloatTensor(w2v.vectors)
#print (weights_text, weights_text.size()) #torch.Size([3000000, 300])
weights_emoji = torch.FloatTensor(e2v.vectors)
#print (weights_emoji, weights_emoji.size()) #torch.Size([1661, 300])

In [6]:
#print(w2v ['Valium'])

In [7]:
###### text preprocessing #########

def remove_string_noise(input_str):
    input_str = re.sub(r"http\S+", "", input_str)
    
    #give special char you want to remove
    #do not put space between chars, and space (" ") is not a special char
    punctuation_noise ="!\"$%&'#()*+,-./:;<=>?@[\]^_`{|}~" #print string.punctuation 
    #number_noise = "0123456789"
    special_noise = ""

    #all_noise = punctuation_noise + number_noise + special_noise
    all_noise = punctuation_noise + special_noise

    for c in all_noise:
        if c in input_str:
            input_str = input_str.replace(c, " ")#replace with space
    fresh_str = ' '.join(input_str.split())
    return fresh_str

def clean_tweets(tweet):
#     #HappyEmoticons
#     emoticons_happy = set([
#         ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
#         ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
#         '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
#         'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
#         '<3'
#         ])

#     # Sad Emoticons
#     emoticons_sad = set([
#         ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
#         ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
#         ':c', ':{', '>:\\', ';('
#         ])

#     #combine sad and happy emoticons
#     emoticons = emoticons_happy.union(emoticons_sad)

    wordnet_lemmatizer = WordNetLemmatizer()
    sw = set(stopwords.words('english'))
    operators = set(('no', 'not', 'nor', 'none'))
    stop_words = set(sw) - operators
    stop_words.update([ 'amp', 'rt'])  ###as we are using set so we used .update....otherwise .extends
    
#     tweet = re.sub(r':', '', tweet)
#     tweet = re.sub(r'‚Ä¶', '', tweet)
#     tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    word_tokens = nltk.word_tokenize(tweet)
    filtered_tweet = []
    for w in word_tokens:
        #if w not in stop_words and w not in emoticons:
        if w not in stop_words:
            filtered_tweet.append(w)
    lemmatized_tweet = []
    for word in filtered_tweet:
        lemmatized_tweet.append(wordnet_lemmatizer.lemmatize(word, pos="v"))

    #print (sentences)
    #return ' '.join(filtered_tweet)
    return ' '.join(lemmatized_tweet)
    #return ' '.join(sentences)
    
def pre_processing_tweets (df):
    #p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.MENTION, p.OPT.RESERVED)
    p.set_options(p.OPT.RESERVED)
    
    clean_text = []
    #for i in range (0, df.shape[0]):
        #clean_text.append(p.clean(str(df['text'][i])))#python 3

    for i in df:
        clean_text.append(p.clean(str(i)))#python 3
    #print(clean_text) #all ok
    fresh_text1 = []
    for i in range (0, df.shape[0]):
        #fresh_text1.append(remove_string_noise(clean_text[i].encode('ascii', 'ignore').decode("utf-8"))) #can remove other emojis and no \UF..
        fresh_text1.append(remove_string_noise(clean_text[i]))
    #print(fresh_text1) #all ok
    
    #Call clean_tweet method-2 for extra preprocessing
    filtered_tweet = []
    for i in range (0, len(fresh_text1)):
        filtered_tweet.append(clean_tweets(fresh_text1[i].lower()))

#     #####for bigram
    
#     df_clean = pd.DataFrame({'clean': filtered_tweet})
#     #df_clean = df_clean.dropna().drop_duplicates()
#     sent = [row.split() for row in df_clean['clean']]
#     #print (sent_word, type(sent_word))

#     ## create bigram
#     #Creates the relevant phrases from the list of sentences:
#     phrases = Phrases(sent, min_count=1, progress_per=10000)
#     #print (phrases, type(phrases))

#     #The goal of Phraser() is to cut down memory consumption of Phrases(), by discarding model state not strictly needed for the bigram detection task:
#     bigram = Phraser(phrases)
#     #print (bigram)

#     #Transform the corpus based on the bigrams detected:
#     sentences = bigram[sent]
#     training_data  = [sentences[i] for i in range(0,len(sentences))]
#     return training_data
    
    return filtered_tweet
    

In [22]:
#Load 15k user location and description

#df = pd.read_csv("data/yoga_user_loc_desc_mentioned_yoga_2600.csv") 
#print (df) #[2597 rows x 4 columns] Name, Location, Description

#Load 1300 user location, description, yoga tweets, utype, umotivation
df = pd.read_csv("data/yoga_user_name_loc_des_mergetweets_yoga_1300_lb.csv") 
#print (df) #[1308 rows x 7 columns] name, location, description, text

#for user description
proc_des = pre_processing_tweets(df['description'])
#print(proc_des, len(proc_des)) #15168


df_clean_des = pd.DataFrame({'clean': proc_des})
#print(df_clean_des.head(2))
token_des = [row.split() for row in df_clean_des['clean']]
#print(token_des, len(token_des)) #2597



In [23]:
#for user location
proc_loc = pre_processing_tweets(df['location'])
#print(proc_loc, len(proc_loc)) #2597

df_clean_loc = pd.DataFrame({'clean': proc_loc})
token_loc= [row.split() for row in df_clean_loc['clean']]
#print(token_loc, len(token_loc)) #2597

In [24]:
from emoji import UNICODE_EMOJI
#print(emoji.UNICODE_EMOJI.keys())
def is_emoji(s):
    count = 0
    for emoji in UNICODE_EMOJI:
    #for bl in emoji.UNICODE_EMOJI.keys():
        #print (bl)
        #print('s.count(bl)', s.count(bl))
        count += s.count(emoji)
        #print('count', count)
        if count > 4:
            return False
    return bool(count)
print (is_emoji("😘")) #true
print(is_emoji("🇺🇸")) #false why??? we can solve it using count > 4 now true
print(is_emoji("🇺🇸love😘"))


True
True
True


In [11]:
def build_vocab_using_word_embedding(training_data, model_w2v):    
    word_to_ix = {}
    for sent in training_data:
        #print ("sent", sent)
        for word in sent:
            #print ("word", word)
            if word not in word_to_ix:
                if word in model_w2v.vocab:
                    word_to_ix[word] = model_w2v.vocab[word].index
                    #print "(word_to_ix", word_to_ix)
    return word_to_ix
def create_word_embeddings_w2v(weights):
    word_embeddings = nn.Embedding.from_pretrained(torch.FloatTensor(weights))
    return word_embeddings

def prepare_sequence_word_em(seq, to_ix):
    idxs = []
    #for w in seq:
    if seq in to_ix:
        idxs.append(to_ix[seq])
    else:
        idxs.append(to_ix["my_unknown"])         

    return torch.tensor(idxs, dtype=torch.long)




In [12]:
#### build word_to_ix and emoji_to_ix for Location

word_to_ix = build_vocab_using_word_embedding(token_loc, w2v)
# Adding a new key value pair for unknown word
word_to_ix.update( {'my_unknown' : len(word_to_ix)} ) 
#print (word_to_ix, len(word_to_ix)) #618

word_embeddings = create_word_embeddings_w2v(weights_text)
#print(word_embeddings) #Embedding(3000000, 300)


emo_to_ix = build_vocab_using_word_embedding(token_loc, e2v)
# Adding a new key value pair for unknown emoji
emo_to_ix.update( {'my_unknown' : len(emo_to_ix)} ) 
#print (emo_to_ix, len(emo_to_ix)) #3

emoji_embeddings = create_word_embeddings_w2v(weights_emoji)
#print(emoji_embeddings) #Embedding(1661, 300)


In [13]:
#### Prepare senquence including text and emoji for whole user location


#token_loc = [['맠동', '잼젠'], ['hyderabad', 'india', '🇮🇳'], ['bharat'], ['india'], ['retweets', 'r', 'not', 'endorsements']]
#print(len(token_loc))#5
#user_loc_embedd = []
user_loc_embedd_d = {}
for i in range(len(token_loc)):
    #print (token_loc[i])
    sentence_embedd = []
    text =[]
    emoji = []
    for j in range(len(token_loc[i])):
        #print (token_loc[i][j])
        if is_emoji(token_loc[i][j]) == False:  
            #print (token_loc[i][j])
            sequence = prepare_sequence_word_em(token_loc[i][j], word_to_ix)
            #print(sequence)
            embeds = word_embeddings(sequence)
            text.append(embeds)
            #text.append(torch.tensor(w2v[i])) #problem with OOV word
            #build the sentence embedding
            sentence_embedd.append(embeds)

        if is_emoji(token_loc[i][j]) == True:
            #print (token_loc[i][j])
            sequence_emo = prepare_sequence_word_em(token_loc[i][j], emo_to_ix)
            #print(sequence)
            embeds_emo = emoji_embeddings(sequence_emo)
            emoji.append(embeds_emo)
            #emoji.append(torch.tensor(e2v[i])) #problem with OOV emoji
            #build the sentence embedding
            sentence_embedd.append(embeds_emo)
    #sys.exit()
        
#     print('len(text):', len(text)) 
#     print('len(emoji):', len(emoji)) 
#     print('len(sentence_embedd)', len(sentence_embedd))
#     #print(sentence_embedd)
    #user_loc_embedd.append(sentence_embedd)
    user_loc_embedd_d[df.name[i]] = sentence_embedd
     
#print('len(user_loc_embedd)', len(user_loc_embedd))  #1308  
# print(text[0])
# print(emoji[0])
#print(e2v['💙'])
# print (user_loc_embedd_d['000mrs000'])
# print (user_loc_embedd_d['1freespirityogi'])
torch.save(user_loc_embedd_d, 'data/locationEmbeddings.pt')

In [14]:
#### build word_to_ix and emoji_to_ix for user Description

word_to_ix = build_vocab_using_word_embedding(token_des, w2v)
# Adding a new key value pair for unknown word
word_to_ix.update( {'my_unknown' : len(word_to_ix)} ) 
#print (word_to_ix, len(word_to_ix)) #3482

word_embeddings = create_word_embeddings_w2v(weights_text)
#print(word_embeddings) #Embedding(3000000, 300)


emo_to_ix = build_vocab_using_word_embedding(token_des, e2v)
# Adding a new key value pair for unknown emoji
emo_to_ix.update( {'my_unknown' : len(emo_to_ix)} ) 
#print (emo_to_ix, len(emo_to_ix)) #132

emoji_embeddings = create_word_embeddings_w2v(weights_emoji)
#print(emoji_embeddings) #Embedding(1661, 300)

In [15]:
#### Prepare senquence including text and emoji for whole user description

#user_des_embedd = []
user_des_embedd_d = {}
for i in range(len(token_des)):
    #print (token_des[i])
    sentence_embedd = []
    #text =[]
    #emoji = []
    for j in range(len(token_des[i])):
        #print (token_des[i][j])
        if is_emoji(token_des[i][j]) == False:  
            #print (token_des[i][j])
            sequence = prepare_sequence_word_em(token_des[i][j], word_to_ix)
            #print(sequence)
            embeds = word_embeddings(sequence)
            #text.append(embeds)
            #text.append(torch.tensor(w2v[i])) #problem with OOV word
            #build the sentence embedding
            sentence_embedd.append(embeds)

        if is_emoji(token_des[i][j]) == True:
            #print (token_des[i][j])
            sequence_emo = prepare_sequence_word_em(token_des[i][j], emo_to_ix)
            #print(sequence)
            embeds_emo = emoji_embeddings(sequence_emo)
            #emoji.append(embeds_emo)
            #emoji.append(torch.tensor(e2v[i])) #problem with OOV emoji
            #build the sentence embedding
            sentence_embedd.append(embeds_emo)
#     print('len(text):', len(text)) 
#     print('len(emoji):', len(emoji)) 
#    print('len(sentence_embedd)', len(sentence_embedd))
#     #print(sentence_embedd)
    #user_des_embedd.append(sentence_embedd)
    user_des_embedd_d[df.name[i]] = sentence_embedd
#print('len(user_des_embedd)', len(user_des_embedd))  #1308
torch.save(user_des_embedd_d, 'data/descriptionEmbeddings.pt')

In [16]:
# def startTraining_des(sentence_in, dimensionsOrg):
#     for i in range(len(sentence_in)):
#         #print (train_lex[i])
#         for j in range(len(sentence_in[i])):
#             trainTuple= sentence_in[i][j].view(1, 300)
#     #print(trainTuple.size()) #torch.Size([1, 300])
#     #trainNeuralNet_option2(trainTuplesConcat,targetTuplesConcat,targetTuplesConcatIndex, dimensionsOrg, tag_dim)


In [17]:
# ###start training description
# dimensionsOrg = 300
# #print(user_des_embedd[1], len(user_des_embedd[1])) #13
# #print(user_des_embedd[1][0], user_des_embedd[1][0].size()) #torch.Size([1, 300])
# #print(token_des[1], len(token_des[1]), token_des[1][0], len(token_des[1][0]))
# startTraining_des(user_des_embedd, dimensionsOrg)


In [18]:
##### Tweets of user (expensive)####

#for user tweets
proc_text = pre_processing_tweets(df['text'])
#print( len(proc_text)) #1308






In [19]:
df_clean_text = pd.DataFrame({'clean': proc_text})
token_text = [row.split() for row in df_clean_text['clean']]
print(len(token_text)) #1308

1308


In [20]:
#### build word_to_ix and emoji_to_ix for user tweets

word_to_ix = build_vocab_using_word_embedding(token_text, w2v)
# Adding a new key value pair for unknown word
word_to_ix.update( {'my_unknown' : len(word_to_ix)} ) 
#print (word_to_ix, len(word_to_ix)) #75781

word_embeddings = create_word_embeddings_w2v(weights_text)
#print(word_embeddings) #Embedding(3000000, 300)


emo_to_ix = build_vocab_using_word_embedding(token_text, e2v)
# Adding a new key value pair for unknown emoji
emo_to_ix.update( {'my_unknown' : len(emo_to_ix)} ) 
#print (emo_to_ix, len(emo_to_ix)) #1409

emoji_embeddings = create_word_embeddings_w2v(weights_emoji)
#print(emoji_embeddings) #Embedding(1661, 300)

In [21]:
#### Prepare senquence including text and emoji for whole user tweets

#user_text_embedd = []
user_text_embedd_d = {}
for i in range(len(token_text)):
    #print (token_text[i])
    sentence_embedd = []
    #text =[]
    #emoji = []
    for j in range(len(token_text[i])):
        #print (token_des[i][j])
        if is_emoji(token_text[i][j]) == False:  
            #print (token_text[i][j])
            sequence = prepare_sequence_word_em(token_text[i][j], word_to_ix)
            #print(sequence)
            embeds = word_embeddings(sequence)
            #text.append(embeds)
            #text.append(torch.tensor(w2v[i])) #problem with OOV word
            #build the sentence embedding
            sentence_embedd.append(embeds)

        if is_emoji(token_text[i][j]) == True:
            #print (token_text[i][j])
            sequence_emo = prepare_sequence_word_em(token_text[i][j], emo_to_ix)
            #print(sequence)
            embeds_emo = emoji_embeddings(sequence_emo)
            #emoji.append(embeds_emo)
            #emoji.append(torch.tensor(e2v[i])) #problem with OOV emoji
            #build the sentence embedding
            sentence_embedd.append(embeds_emo)
#     print('len(text):', len(text)) 
#     print('len(emoji):', len(emoji)) 
#    print('len(sentence_embedd)', len(sentence_embedd))
#     #print(sentence_embedd)
    #user_text_embedd.append(sentence_embedd)
    user_text_embedd_d[df.name[i]] = sentence_embedd
    
    
#print('len(user_text_embedd)', len(user_text_embedd))  #1308
torch.save(user_text_embedd_d, 'data/tweetsEmbeddings.pt')